# KNN
## 개요
KNN 은 K-Nearest Neighbor 를 뜻하며 K개의 인접 데이터를 통해 문제를 해결하는 방법이다. 다른 회귀 모델들과 달리 Instance-based Learning 으로 분류된다. 회귀와 분류 두 문제에 대해 모두 쓸 수 있다.

### **Model-based Learning**
- 데이터로부터 모델을 생성하여 문제 해결
- Linear Regression
- Logistic Regression
- Neural Network

### **Instance-based Learning**
- 모델 생성하지 않고 예측 하려는 데이터의 인접 데이터만을 이용하여 문제 해결
- KNN
- Locally Weighted Regression

## 방법
### 거리
거리 기반의 알고리즘이기 때문에 거리 함수를 정하는 것이 중요하다. 다음은 잘 알려진 몇개의 거리 함수이다.

1. Euclidean Distance

$$ d_{ij} = \sqrt{\sum (x_{ik} - x_{jk})^2}$$

2. Manhattan Distance

$$ d_{ij} = \sum |x_{ik} - x_{jk}| $$

위 두 거리 함수를 일반화 하면 **Minkowski Distance** 를 사용 할 수 있다.
$$ d_{ij} = (\sum |x_{ik} - x_{jk}|^r)^{1/r} $$
$r=1$ (L1 Norm) 일 때는 Manhattan, $r=2$ (L2 Norm) 일 때는 Euclidean, $r=\infty$ 일 때는 **Supremum Distance**.

이 외에도 타원형 거리인 **Mahalanobis Distance**, 두 데이터 사이의 상관계수인 **Correlation Distance**, Spearman Rank Correlation을 이용하는 **Spearman Distance** 가 있다.

문제에 따라 적절한 거리를 사용하는 것이 알고리즘의 성능을 높일 수 있다.

### 예측값 계산
1. Average/Majority Voting

거리 함수를 이용 하여 예측할 데이터와 가장 가까운 K개의 데이터를 구한다. 만약 회귀 문제라면 구한 K개의 데이터의 레이블의 평균값, 분류 문제라면 가장 많은 카테고리로 분류한다.

2. Weighted KNN

예측할 데이터에 가까울 수록 가중치를 주어 예측에 더 큰 영향을 줄 수 있게 하는 방법이다. 예를 들어 예측할 데이터 $X_test$와 가장 가까운 데이터 K개를 $X_1, \ldots, X_k$, 각 거리를 $d_1, \ldots, d_k$, 각 데이터에 대한 레이블을 $y_1, \ldots, y_k$ 라 하자. 그렇다면 $X_test$ 와 $X_i$ 사이의 **유사도** $s_i=1/d_i$로 정의 할 수 있다.

따라서 가중치를 적용한 예측값은 다음과 같다.
$$ y_{test}=\frac{\sum s_i y_i}{\sum s_i} $$

### 주의사항
거리 계산 할 때 모든 요소에 대한 단위가 같아야 하므로 표준화 하는 것이 특히 중요하다.

분류 문제의 경우 One-Hot Encoding 을 해 주어야 한다.

#### One-Hot Encoding
모든 값이 수치로 되어 있어야 하기 때문에 분류 문제의 경우 레이블을 One-Hot Encoding 을 해주어야 한다. 즉, 어떤 데이터 $X_i$가 $j$번째 카테고리로 분류 되어 있다면, $\hat{y}_{ij}=1, \hat{y}_{ik}=0, k \neq j$로 정의하는 것이다. 예를 들어, 5개의 카테고리로 분류되어 있고, 이에 따른 데이터셋의 레이블이 다음과 같다고 해보자.
$$ y=\begin{bmatrix}3 \\ 1 \\ 0 \\ 2 \\ 1 \\ 4\end{bmatrix} $$
이러한 $y$ 를 One-Hot Encoding 해 주면 다음으로 바뀐다.

$$ \hat{y}=\begin{bmatrix}
0 & 0 & 0 & 1 & 0 \\
0 & 1 & 0 & 0 & 0 \\
1 & 0 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 & 0 \\
0 & 1 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 1\end{bmatrix} $$

## 실습: Iris 데이터 분류

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.datasets import load_iris

In [ ]:
iris = load_iris()

In [ ]:
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
df['target'] = iris.target

df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [ ]:
X_tr, X_tst, y_tr, y_tst = train_test_split(df.drop('target', axis=1), df['target'],  test_size=0.3)
X_tr, X_tst, y_tr, y_tst = map(lambda x: np.array(x), [X_tr, X_tst, y_tr, y_tst])

X_tr.shape, X_tst.shape

((105, 4), (45, 4))

In [ ]:
def standardize(X):
    return (X - X.mean()) / X.std()

def normed_dist(X, X_i, r):
    s = np.sum(np.power(np.abs(X - X_i), r), axis=1)
    return np.power(s, 1/r)

def one_hot(y, c):
    r = y.shape[0]
    y_hat = np.zeros((r, c))
    y_hat[np.arange(r), y] = 1
    return y_hat

def KNNClassifier(X, y, X_tst, K, r, weighted=False):
    y_predict = np.zeros(X_tst.shape[0])
    c = np.max(y) + 1
    i = 0
    for row in X_tst:
        d = normed_dist(row, X, r)
        args = np.argsort(d)[:K]
        X_k = X[args, :]
        y_hat = one_hot(y[args], c)
        w = 1 / d[args] if weighted else None
        y_predict[i] = np.argmax(np.average(y_hat, axis=0, weights=w))
        i += 1
    return y_predict

def KNNRegression(X, y, X_tst, K, r, weighted=False):
    y_predict = np.zeros(X_tst.shape[0])
    c = np.max(y) + 1
    i = 0
    for row in X_tst:
        d = normed_dist(row, X, r)
        args = np.argsort(d)[:K]
        X_k = X[args, :]
        y_hat = y[args]
        w = 1 / d[args] if weighted else None
        y_predict[i] = np.average(y_hat, weights=w)
        i += 1
    return y_predict

def acc_score(y, y_hat):
    return np.sum(y == y_hat) / y.shape[0]

def mse(y, y_hat):
    sse = np.power(np.abs(y - y_hat), 2)
    return np.mean(np.power(sse, 1/2))

In [ ]:
K = 3

X_tr = standardize(X_tr)
X_tst = standardize(X_tst)

naive_l1 = KNNClassifier(X_tr, y_tr, X_tst, K, 1)
naive_l2 = KNNClassifier(X_tr, y_tr, X_tst, K, 2)
weighted_l1 = KNNClassifier(X_tr, y_tr, X_tst, K, 1, weighted=True)
weighted_l2 =  KNNClassifier(X_tr, y_tr, X_tst, K, 2, weighted=True)

acc_score(y_tst, naive_l1), acc_score(y_tst, naive_l2),  acc_score(y_tst, weighted_l1),  acc_score(y_tst, weighted_l2)

(0.9777777777777777,
 0.9777777777777777,
 0.9777777777777777,
 0.9777777777777777)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knn = KNeighborsClassifier(n_neighbors=K)
knn.fit(X_tr, y_tr)
y_hat = knn.predict(X_tst)

acc_score(y_tst, y_hat)

0.9777777777777777

## 실습: 보스턴 집값 회귀

In [ ]:
from sklearn.datasets import load_boston

In [ ]:
boston = load_boston()

In [ ]:
df = pd.DataFrame(data=boston.data, columns=boston.feature_names)
df['target'] = boston.target

df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [ ]:
X_tr, X_tst, y_tr, y_tst = train_test_split(df.drop('target', axis=1), df['target'],  test_size=0.3)
X_tr, X_tst, y_tr, y_tst = map(lambda x: np.array(x), [X_tr, X_tst, y_tr, y_tst])

X_tr.shape, X_tst.shape

((354, 13), (152, 13))

In [ ]:
K = 3

X_tr = standardize(X_tr)
X_tst = standardize(X_tst)

for K in range(1, 10):
    naive_l1 = KNNRegression(X_tr, y_tr, X_tst, K, 1)
    naive_l2 = KNNRegression(X_tr, y_tr, X_tst, K, 2)
    weighted_l1 = KNNRegression(X_tr, y_tr, X_tst, K, 1, weighted=True)
    weighted_l2 =  KNNRegression(X_tr, y_tr, X_tst, K, 2, weighted=True)

    print(K, mse(y_tst, naive_l1), mse(y_tst, naive_l2),  mse(y_tst, weighted_l1), mse(y_tst, weighted_l2))

1 3.954605263157894 4.451315789473684 3.954605263157894 4.451315789473685
2 3.6131578947368426 4.24703947368421 3.4987438073079176 4.061681854664688
3 3.3528508771929824 3.736842105263158 3.23271855999984 3.68132380396935
4 3.317763157894737 3.5993421052631582 3.218718161401797 3.4718208691962062
5 3.4113157894736843 3.7442105263157894 3.2199274405892675 3.525408042883383
6 3.4303728070175445 3.856140350877193 3.2342208641335692 3.548474545686575
7 3.514003759398496 3.9962406015037586 3.2649162438355424 3.6513707474683454
8 3.6014802631578946 4.043832236842104 3.2906643919801786 3.6760585464903026
9 3.729093567251462 4.086769005847953 3.380195585017291 3.681194601349201


In [ ]:
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
for K in range(1, 10):
    knn = KNeighborsRegressor(n_neighbors=K)
    knn.fit(X_tr, y_tr)
    y_hat = knn.predict(X_tst)

    print(K, mse(y_tst, y_hat))

1 0.08888888888888889
2 0.05555555555555555
3 0.05185185185185184
4 0.05555555555555555
5 0.06666666666666665
6 0.07407407407407407
7 0.07301587301587302
8 0.06944444444444445
9 0.06666666666666668
